## Load library and stuff

In [ ]:
!nvidia-smi

In [ ]:
import os
import csv
import sys
import tensorflow as tf
import keras
import keras.preprocessing.image
import json
import random
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math

In [ ]:
# keras_retinanet imports
from keras_retinanet import models
from keras_retinanet import losses
from keras_retinanet import layers
from keras_retinanet.models.retinanet import retinanet_bbox

from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# sys.path.append('/root/amol/product_detection/keras-retinanet/keras_retinanet/preprocessing/')
# from csv_generator import CSVGenerator
# from ..models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
# from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet


from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.keras_version import check_keras_version
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
# from eval_modified import evaluate

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [ ]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

## create train and val set

In [ ]:
import random
from collections import Counter
SEED = 334

In [ ]:
dataset = []
with open('/root/data/pellet/labels/pellet_labels.csv', 'r') as f:
    reader = csv.reader(f)
    for l in reader:
        dataset.append(l)

In [ ]:
dataset = sorted(dataset, key=lambda k:k[0])

In [ ]:
print(len(dataset))

In [ ]:
random.seed(SEED)
random.shuffle(dataset)
train = dataset[:4500]
val = dataset[500:]

In [ ]:
with open('/root/data/pellet/labels/pellet_labels_train.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for bbox in train:
        writer.writerow(bbox)

In [ ]:
with open('/root/data/pellet/labels/pellet_labels_val.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for bbox in val:
        writer.writerow(bbox)

# Create generators

In [ ]:
# inputs
args = {"batch_size": 4,
               "config": None,
               "image_min_side": 800,
               "image_max_side": 1300,
               "preprocess_image": True,
               "classes": "/root/data/pellet/labels/class_ids.csv",
               "annotations": '/root/data/pellet/labels/pellet_labels_train.csv',
               "val_annotations": '/root/data/pellet/labels/pellet_labels_val.csv',
               "random_transform": True}

In [ ]:
backbone = models.backbone('resnet50')

In [ ]:
def create_generators(args, preprocess_image):
    """ Create generators for training and validation.
    Args
        args             : parseargs object containing configuration for generators.
        preprocess_image : Function that preprocesses an image for the network.
    """
    common_args = {
        'batch_size'       : args["batch_size"],
        'config'           : args["config"],
        'image_min_side'   : args["image_min_side"],
        'image_max_side'   : args["image_max_side"],
        'preprocess_image' : preprocess_image,
    }

    # create random transform generator for augmenting training data
    if args["random_transform"]:
        transform_generator = random_transform_generator(
            min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.9, 0.9),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,
        )
    else:
        transform_generator = random_transform_generator(flip_x_chance=0.5)
    
    # create the genereators
    train_generator = CSVGenerator(
        args["annotations"],
        args["classes"],
        transform_generator=transform_generator,
        **common_args
    )

    if args["val_annotations"]:
        validation_generator = CSVGenerator(
            args["val_annotations"],
            args["classes"],
            **common_args
        )
    else:
        validation_generator = None

    return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = create_generators(args, backbone.preprocess_image)

# LOAD MODEL

In [ ]:
def model_with_weights(model, weights, skip_mismatch):
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model

In [ ]:
def create_models(backbone_retinanet, num_classes, weights, multi_gpu=0,
                  freeze_backbone=False, lr=1e-5, config=None):

    modifier = freeze_model if freeze_backbone else None

    # load anchor parameters, or pass None (so that defaults will be used)
    anchor_params = None
    num_anchors   = None
    if config and 'anchor_parameters' in config:
        anchor_params = parse_anchor_parameters(config)
        num_anchors   = anchor_params.num_anchors()

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        from keras.utils import multi_gpu_model
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)
    else:
        model          = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=lr, clipnorm=0.001)
    )

    return model, training_model, prediction_model

In [ ]:
weights = backbone.download_imagenet()
# weights = '/root/data/models/gopro/detection/detection_02.h5'

In [ ]:
model, training_model, prediction_model = create_models(backbone_retinanet=backbone.retinanet,
                                                        num_classes=train_generator.num_classes(),
                                                        weights=weights)

# TRAINING

In [ ]:
import json

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 1e-5
    drop = 0.5
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)
        


In [ ]:
saveh = SaveHistory('./detection_fg_history.json')

In [ ]:
# save model
filepath = os.path.join('/root/data/pellet/models/', 'model_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             save_best_only=True, 
                             mode='min')

In [ ]:
# start training
history = training_model.fit_generator(
        generator=train_generator,
        steps_per_epoch=len(train)//args["batch_size"],
        epochs=50,
        verbose=1,
        validation_data= validation_generator,
        validation_steps= len(val) // args["batch_size"],
        callbacks=[saveh, checkpoint]
    )

In [ ]:
training_model.save(("/root/data/pellet/model_0.h5"))

# PLOT HISTORY

In [ ]:
import json
import matplotlib.pyplot as plt

In [ ]:
history = json.load(open('./detection_history.json'))

In [ ]:
plt.plot(history['loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()

# evalute

In [ ]:
training_model.save('train.h5')
prediction_model.save('pred.h5')

In [ ]:
evaluate(validation_generator, prediction_model, score_threshold=0.5)